# 🧠 Kernel: python3

In [ ]:
# ⚙️ Resolución robusta de rutas
from pathlib import Path
def resolve_repo_root(start: Path) -> Path:
    cur = start
    for _ in range(6):
        if (cur / 'data').exists() and (cur / 'notebooks').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start

root = resolve_repo_root(Path.cwd())
raw_path = root / 'data' / 'raw'
processed_path = root / 'data' / 'processed'
processed_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# 📦 Generación de datos sintéticos si no existen
import pandas as pd
import numpy as np
np.random.seed(42)

weeks = pd.date_range('2025-01-06', periods=12, freq='W-MON')
families = ['A', 'B', 'C']
skus = [f'{fam}{i:02d}' for fam in families for i in range(1, 6)]

# Demanda semanal por SKU (base)
dem_base = []
for sku in skus:
    fam = sku[0]
    mean = {'A': 120, 'B': 80, 'C': 60}[fam]
    std = {'A': 20, 'B': 15, 'C': 10}[fam]
    vals = np.clip(np.random.normal(mean, std, len(weeks)).round(), 0, None)
    for w, v in zip(weeks, vals):
        dem_base.append({'week': w, 'family': fam, 'sku': sku, 'demand': int(v)})
df_dem_base = pd.DataFrame(dem_base)

# Capacidad por período (planta + transporte)
df_capacity = pd.DataFrame({
  'week': weeks,
  'prod_capacity': [900, 900, 950, 950, 1000, 1000, 1000, 1000, 950, 950, 900, 900],
  'trans_capacity': [800, 800, 850, 850, 900, 900, 900, 900, 850, 850, 800, 800]
})
df_capacity['total_capacity'] = df_capacity['prod_capacity'] + df_capacity['trans_capacity']

# Fill rate objetivo por familia
df_policies = pd.DataFrame({
  'family': families,
  'fill_rate_target': [0.95, 0.90, 0.88]
})

df_dem_base.to_parquet(processed_path / 'sop_dem_base.parquet', index=False)
df_capacity.to_parquet(processed_path / 'sop_capacity.parquet', index=False)
df_policies.to_parquet(processed_path / 'sop_policies.parquet', index=False)
df_dem_base.head(), df_capacity.head(), df_policies

## 🎛️ Definición de escenarios
source
dem_week_family = df_dem_scen.groupby(['scenario', 'week', 'family'], as_index=False)['demand'].sum()

# Simulación semana a semana
priority = ['A', 'B', 'C']
records = []
for scen in dem_week_family['scenario'].unique():
    backlog = {fam: 0 for fam in priority}
    for row in df_capacity.itertuples(index=False):
        week = row.week
        cap = row.total_capacity
        week_dem = dem_week_family[(dem_week_family['scenario'] == scen) & (dem_week_family['week'] == week)].set_index('family')['demand'].to_dict()
        # demanda + backlog previo
        need = {fam: week_dem.get(fam, 0) + backlog[fam] for fam in priority}
        delivered = {fam: 0 for fam in priority}
        # asignar capacidad por prioridad
        remaining = cap
        for fam in priority:
            take = min(need[fam], remaining)
            delivered[fam] = take
            remaining -= take
        # actualizar backlog
        for fam in priority:
            backlog[fam] = max(need[fam] - delivered[fam], 0)
        util = (cap - remaining) / cap if cap > 0 else 0
        for fam in priority:
            dem = week_dem.get(fam, 0)
            fill_rate = delivered[fam] / dem if dem > 0 else 1.0
            records.append({
,